In [61]:
#훈련세트(약 60%) / 검증세트(약 20%) / 테스트세트 (약 20%)

In [62]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [63]:
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']

In [64]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [65]:
#훈련세트와 검정세트 만들기

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [66]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [67]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [68]:
#3-폴드 교차 검증 => 훈련 세트를 세 부분으로 나눠서 검증을 수행하는 것

In [69]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00551915, 0.00601482, 0.00550961, 0.00551462, 0.00604033]), 'score_time': array([0.00100517, 0.00051808, 0.00251365, 0.00099874, 0.00199461]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [70]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [71]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [72]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [73]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [74]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=1)  #n_jobs = -1로 할 경우 오류가 나서 1로 사용

In [76]:
gs.fit(train_input, train_target)

,estimator,DecisionTreeC...ndom_state=42)
,param_grid,"{'min_impurity_decrease': [0.0001, 0.0002, ...]}"
,scoring,None
,n_jobs,1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'gini'


In [77]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [78]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [79]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [80]:
print(gs.cv_results_['params'][gs.best_index_])

{'min_impurity_decrease': 0.0001}


In [81]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001)
         ,'max_depth': range(5, 20, 1)
         ,'min_samples_split': range(2, 100, 10)
         }

In [82]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=1)
gs.fit(train_input, train_target)

,estimator,DecisionTreeC...ndom_state=42)
,param_grid,"{'max_depth': range(5, 20), 'min_impurity_decrease': array([0.0001... 0.0009]), 'min_samples_split': range(2, 100, 10)}"
,scoring,None
,n_jobs,1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'gini'


In [83]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}


In [84]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [85]:
from scipy.stats import uniform, randint

In [86]:
rgen = randint(0, 10)
rgen.rvs(10)

array([8, 3, 9, 2, 6, 3, 4, 3, 1, 3])

In [87]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 92, 107, 104, 100, 100, 107, 110,  96,  87,  97]))

In [88]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.13105617, 0.64098854, 0.4796858 , 0.18391005, 0.91490039,
       0.97657815, 0.91433771, 0.41110557, 0.44858027, 0.69238503])

In [89]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001)
         ,'max_depth': randint(20,50)
         ,'min_samples_split': randint(2,25)
         ,'min_samples_leaf': randint(1,25)
         }

In [90]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=1, random_state=42)
rs.fit(train_input, train_target)

,estimator,DecisionTreeC...ndom_state=42)
,param_distributions,"{'max_depth': <scipy.stats....001B508983A90>, 'min_impurity_decrease': <scipy.stats....001B508980050>, 'min_samples_leaf': <scipy.stats....001B508983850>, 'min_samples_split': <scipy.stats....001B508981390>}"
,n_iter,100
,scoring,None
,n_jobs,1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [91]:
print(rs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}


In [92]:
print(np.max(rs.cv_results_['mean_test_score']))

0.8695428296438884


In [93]:
dt = rs.best_estimator_
print(dt.score(test_input, test_target))

0.86


In [ ]:
#매개변수는 그리드 서치나 랜덤 서치를 이용해서 쉽게 바꾸자!!